In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
# from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.utils.data import DataLoader
from torchmetrics.text import BLEUScore
from transformers.optimization import AdamW
from tqdm.notebook import tqdm_notebook
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
import numpy as np

/Users/john_gold/Desktop/assigmnt_final/SQuAD_Azerbaijan/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
writer = SummaryWriter('experiments_results')

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/xlm-roberta-large-squad2"


# nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
# QA_input = {
#     'question': 'Beyons neçə yaşında LaTavia Robertsonla tanış oldu?',
#     'context': """Səkkiz yaşında, Beyonce və uşaqlıq dostu Kelly Rowland, bütün qızlardan ibarət əyləncə qrupunun dinləmələrində LaTavia Roberson ilə tanış oldular. Onlar Girl's Tyme kimi daha üç qızla bir qrupa yerləşdirildilər və Hyustondakı istedad şousunda rap etdi və rəqs etdilər. Qrupu gördükdən sonra R&B prodüseri Arne Frager onları Şimali Kaliforniya studiyasına gətirdi və o zamanlar milli televiziyada ən böyük istedad şousu olan Star Search-ə yerləşdirdi. Girl's Tyme qalib gələ bilmədi və Beyonce daha sonra ifa etdikləri mahnının yaxşı olmadığını söylədi. 1995-ci ildə Beyonsun atası qrupu idarə etmək üçün işindən istefa verdi. Bu addım Beyons ailəsinin gəlirini yarıbayarı azaltdı və valideynləri ayrı mənzillərə köçmək məcburiyyətində qaldılar. Mathew orijinal heyəti dördə kəsdi və qrup digər qurulmuş R&B qız qrupları üçün açılış aktı kimi çıxış etməyə davam etdi. Qızlar səsyazma şirkətlərindən əvvəl dinləmələrdən keçdilər və nəhayət, Elektra Records ilə müqavilə imzaladılar, ilk səsyazmaları üzərində işləmək üçün qısa müddətə Atlanta Records-a köçdülər, ancaq şirkət tərəfindən kəsildi. Bu, ailəni daha da gərginləşdirdi və Beyonsun valideynləri ayrıldı. 5 oktyabr 1995-ci ildə Dwayne Wiggins-in Grass Roots Entertainment qrupu ilə müqavilə imzaladı. 1996-cı ildə qızlar Sony Music ilə razılaşma əsasında debüt albomlarını yazmağa başladılar, Knowles ailəsi yenidən birləşdi və qısa müddət sonra qrup Columbia Records ilə müqavilə bağladı.	
# """
# }
# res = nlp(QA_input)


model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device('mps')
print(device)
model = model.to(device)
# tokenizer = tokenizer.to(device)

mps


In [5]:
model

XLMRobertaForQuestionAnswering(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_feature

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

questions = [
    """"19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?"""]

text = r"""Qalan qrup üzvləri 2000-ci ildə çəkilmiş Charlie's Angels filminin soundtrackində görünən "Müstəqil Qadınlar Part I" mahnısını yazdılar. Ardıcıl on bir həftə ərzində ABŞ-ın Billboard Hot 100 cədvəlində birinci yeri tutaraq, onların ən yaxşı chart olan sinqlı oldu. 2001-ci ilin əvvəlində, Destiny's Child üçüncü albomunu tamamlayarkən, Beyonce amerikalı aktyor Mekhi Phifer ilə birlikdə MTV-nin televiziya üçün hazırlanmış "Carmen: A Hip Hopera" filmində böyük rol aldı. Filadelfiyada cərəyan edən film fransız bəstəkarı Georges Bizetin 19-cu əsrdə “Karmen” operasının müasir şərhidir. 2001-ci ilin may ayında üçüncü "Survivor" albomu çıxanda Luckett və Roberson mahnıların onlara yönəldiyini iddia edərək məhkəməyə müraciət etdilər. Albom ilk həftədə 663.000 nüsxə satışı ilə ABŞ Billboard 200-də bir nömrədə debüt etdi. Albom digər bir nömrəli hitləri, "Bootylicious" və baş treki olan "Survivor"u doğurdu, sonuncusu qrupa Duo və ya Vokallı Qrup tərəfindən Ən Yaxşı R&B Performansı üçün Qremmi Mükafatı qazandırdı. 2001-ci ilin oktyabrında 8 Days of Christmas adlı bayram albomunu buraxdıqdan sonra qrup solo karyeralarını davam etdirmək üçün fasilə elan etdi."""



for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt").to("mps")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(inputs.keys())
    res = model(**inputs)
   
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.cpu().detach().numpy(), -1), np.expand_dims(end.cpu().detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print(f"Score: {scores}\n")

dict_keys(['input_ids', 'attention_mask'])
Question: "19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?
Answer: Georges Bizetin

Score: [0.99064815]



In [7]:
from datasets import load_dataset

In [8]:
dataset = load_dataset('vrashad/squad_azerbaijan')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import pandas as pd

In [78]:
df_train = pd.DataFrame(dataset['train'],columns=dataset['train'].features)
df_test = pd.DataFrame(dataset['test'],columns=dataset['test'].features)

In [79]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [80]:
df_test.head()

,id,title,context,question,answer_text,answer_start,is_impossible
0,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
1,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
2,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
3,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
4,56ddde6b9a695914005b9629,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandlar nə vaxt Normandiyada idilər?,10 və 11-ci əsrlər,94.0,False


In [81]:
def get_index_of_word_in_sentence(context,answer,answer_start):
    if answer is not None:
        word = answer.split()[0].lower()
        try:
            answer_start_new = context.lower().index(word)
            return {'answer_start':[int(answer_start_new)],'text':[answer]}
        except:
            return {'answer_start':[int(answer_start)],'text':[answer]}
    else:
        return {'answer_start':[],'text':[]}

df_train['answers']= df_train.apply(lambda x: get_index_of_word_in_sentence(x.context,x.answer_text,x.answer_start),axis=1)
df_train.drop(columns=['answer_text','answer_start'],inplace=True)

df_test['answers']= df_test.apply(lambda x: get_index_of_word_in_sentence(x.context,x.answer_text,x.answer_start),axis=1)
df_test.drop(columns=['answer_text','answer_start'],inplace=True)
# df['answers']= df.apply(lambda x: print(x.question),axis=1)

  0%|          | 0/1572 [01:41<?, ?it/s]


In [82]:
df_train

,id,title,context,question,is_impossible,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt populyarlaşmağa başladı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons böyüyərkən hansı sahələrdə yarışırdı?,False,"{'answer_start': [203], 'text': ['mahnı oxumaq..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt Destiny's Child-dən ayrılaraq ...,False,"{'answer_start': [534], 'text': ['2003']}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı şəhərdə və əyalətdə böyüdü?,False,"{'answer_start': [166], 'text': ['Hyuston, Tex..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı onillikdə məşhurlaşdı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
...,...,...,...,...,...,...
130046,5735d259012e2f140011a09d,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandu ilk dəfə ABŞ-ın hansı ştatında beynəl...,False,"{'answer_start': [173], 'text': ['Oreqon']}"
130047,5735d259012e2f140011a09e,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Yanqon əvvəllər nə kimi tanınırdı?,False,"{'answer_start': [292], 'text': ['Ranqun']}"
130048,5735d259012e2f140011a09f,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandunun Belarusun hansı şəhəri ilə əlaqəsi...,False,"{'answer_start': [342], 'text': ['Minsk']}"
130049,5735d259012e2f140011a0a0,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandu ilk beynəlxalq əlaqələri neçənci ildə...,False,"{'answer_start': [153], 'text': ['1975']}"


In [83]:
from datasets import Dataset, DatasetDict

In [84]:
datasets_n = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })

In [85]:
datasets_n

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'is_impossible', 'answers', '__index_level_0__'],
        num_rows: 86820
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'is_impossible', 'answers', '__index_level_0__'],
        num_rows: 20302
    })
})

In [139]:
batch_size = 16
max_length = 512 
doc_stride = 56
pad_on_right = tokenizer.padding_side == "right"

In [140]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
 


    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [141]:
df_train[:10000]

,id,title,context,question,is_impossible,answers
0,5727746d708984140094de08,Switzerland,"1200-cü ilə qədər İsveçrə yaylası Savoy, Zähri...",1264-cü ildə Kiburq sülaləsinin süqutuna səbəb...,False,"{'answer_start': [294], 'text': ['1263-cü ildə..."
1,571098c9a58dae1900cd6aaa,Age_of_Enlightenment,Qəhvəxanalar Maarifçilik dövründə biliyin yayı...,Hansı məkan müxtəlif təbəqələrdən olan insanla...,False,"{'answer_start': [0], 'text': ['Qəhvəxanalar']}"
2,56cef84faab44d1400b88d56,New_York_City,"Şəhərin ümumi sahəsi 468,9 kvadrat mil (1,214 ...",Nyu Yorkun ən hündür nöqtəsinin adı nədir?,False,"{'answer_start': [175], 'text': ['Todt Hill']}"
3,57315c6c497a881900248e82,Mosaic,Konstantin Monomaxosun digər böyük işi 1042-10...,Rus abbat Daniel 1106-07-ci illərdə hara səyah...,False,"{'answer_start': [270], 'text': ['Yerusəlim']}"
4,570d4d9efed7b91900d45e50,Valencia,15-ci əsrdə günbəz əlavə edildi və neflər xoru...,Həvarilərin qapısı hansı üslubdadır?,False,"{'answer_start': [411], 'text': ['Qotika']}"
...,...,...,...,...,...,...
9867,56bfb8dca10cfb140055127b,Beyoncé,"O, Destiny's Child ilə yazılan mahnıların əksə...",Jay Z ilə onun yeni mövzuları nələr idi?,False,"{'answer_start': [309], 'text': ['insana qullu..."
9868,57301e6bb2c2fd14005688bd,Roman_Republic,Klodius şəhəri dəhşətə gətirən silahlı dəstələ...,Pompey ardıcıllarına edilən hücumlara kim cava...,False,"{'answer_start': [0], 'text': ['Clodius']}"
9869,56db76b7e7c41114004b5154,2008_Summer_Olympics_torch_relay,"Xinhua və CCTV, əksər Qərb mediasından daha ço...",Hansı nəşr etirazçıların insan haqlarını tapda...,False,"{'answer_start': [592], 'text': ['Sinxua']}"
9870,5728cdc64b864d1900164e75,Detroit,Bölgənin bir çox görkəmli muzeyləri Ueyn Dövlə...,MOCAD nə deməkdir?,False,"{'answer_start': [171], 'text': ['Detroit Müas..."


In [142]:


train = df_train[:10000].sample(frac=1, random_state=42)


df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [143]:
train_dataset[0]


{'id': '56e758c700c9c71400d76fe5',
 'title': 'Paper',
 'context': 'Vərəqə basaraq suyu güclə çıxarır; su təbəqədən zorla çəkildikdən sonra suyu toplamaq üçün ənənəvi ilə qarışdırılmayan xüsusi növ keçə istifadə olunur; halbuki əl ilə kağız hazırlayarkən onun yerinə qurutma vərəqi istifadə olunur.',
 'question': 'Presdən xaric olan suyu toplamaq üçün nə istifadə olunur?',
 'is_impossible': False,
 'answers': {'answer_start': [106], 'text': ['hiss etdi']}}

In [144]:
train_dataset[0]['context'][445:]

''

In [145]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)












Map: 100%|██████████| 9808/9808 [00:02<00:00, 4875.22 examples/s]

Map: 100%|██████████| 64/64 [00:00<00:00, 4102.07 examples/s]


In [146]:
tokenized_valid_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 64
})

In [153]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="finetune-BERT-squad",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [154]:
device = torch.device('mps')
device

device(type='mps')

In [155]:
if torch.backends.mps.is_built():
    print("PyTorch is built with MPS support")
else:
    print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")

PyTorch is built with MPS support


In [156]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [157]:
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [158]:
import torch

trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 13.69 GB, other allocations: 4.23 GB, max allowed: 18.13 GB). Tried to allocate 512.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).